In [38]:
import ollama

In [39]:
message = "tell me a joke"
prompt = {"role": "user", "content":message}
response = ollama.chat(
    model = "qwen2.5:1.5b",
    messages = [prompt])

print(response.message.content)

Sure! Here's a classic one:

Why did the tomato turn red?

Because it saw the salad dressing!

I hope that made you laugh! Let me know if you want another joke or any other help from my side.


In [40]:
class Agent:
    def __init__(self, system = ""):
        self.system = system 
        self.message = []
        if self.system:
            self.message.append({"role":"system", "content":self.system})
    
    def __call__(self, prompt):
        self.message.append({"role":"user", "content":prompt})
        result = self.execute()
        self.message.append({"role":"assistant", "content":result})
        return result
    
    def execute(self, model = "qwen2.5:1.5b"):
        response = ollama.chat(
            model = model,
            messages=self.message
        )
        return response.message.content
    


In [41]:
agent = Agent()

In [42]:
print(agent("come with more creative"))

Sure! Here are some creative ideas:

  * How about we have an impromptu paint and sip night where everyone brings their favorite snacks and drinks to create art in the style of one of your favorite painters?
  * Another idea could be a movie trivia game where each team takes turns answering questions based on their favorite movies. The team that has the most correct answers wins!
  * For an escape room adventure, gather a group of friends or family members for an immersive experience. Find clues around the house or in secret locations and work together to solve puzzles and complete challenges.
  * How about organizing a virtual dance party where everyone can join from home? Dress up in your favorite costume or wear pajamas and dance along to your favorite songs!
  * For something more adventurous, organize a scavenger hunt with clues around the neighborhood. The first person who finds all the answers wins a small prize.
  * Another fun idea could be hosting a movie marathon where every

In [43]:
prompt = '''
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_cost:
e.g. get_cost: book
returns the cost of a book

wikipedia:
e.g. wikipedia: LangChain
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session #1:

Question: How much does a pen cost?
Thought: I should look the pen cost using get_cost
Action: get_cost: pen
PAUSE

You will be called again with this:

Observation: A pen costs $5

You then output:

Answer: A pen costs $5


Example session #2

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
'''.strip()


## Creating Tools

In [44]:
def Calculator(question):
    """This tool is used to solve the math problems such as 2**5  or 34*56 and such type"""
    return eval(question)

In [45]:
Calculator("34**24")

5695003679558247880375471569828315136

In [46]:
# 2. the get_cost() function returns the cost for a pen, a book, and a stapler
def get_cost(thing):
    '''
    This function returns the cost for a pen, a book, and a stapler
    '''
    if thing in 'pen': 
        return('A pen costs $5')
    elif thing in 'book':
        return('A book costs $20')
    elif thing in 'stapler':
        return('A stapler costs $10')
    else:
        return('A random thing for writing costs $12.')

In [47]:
get_cost("pen")

'A pen costs $5'

In [48]:
import re
import httpx

# 3. the wikipedia() function uses the Wikipedia API to search for a specific query on Wikipedia
def wikipedia(q):
    '''
    the wikipedia() function uses the Wikipedia API to search for a specific query on Wikipedia
    '''
    response = httpx.get('https://en.wikipedia.org/w/api.php', params={
        'action': 'query',
        'list': 'search',
        'srsearch': q,
        'format': 'json'
    })
    results = response.json().get('query').get('search', [])
    
    if not results:
        return None
    return results[0]['snippet']

In [49]:
result = wikipedia("Shivaji Maharaj")

In [50]:
from IPython.display import Markdown
Markdown(result)

<span class="searchmatch">Shivaji</span> I (<span class="searchmatch">Shivaji</span> Shahaji Bhonsale, Marathi pronunciation: [ʃiˈʋaːdʑiː ˈbʱos(ə)le]; c. 19 February 1630 – 3 April 1680) was an Indian ruler and a member

In [51]:
# dictionary that maps the function names to the functions themselves
known_actions = {
    'calculate': Calculator,
    'get_cost': get_cost,
    'wikipedia': wikipedia
}

### Testing Agent

In [52]:
my_agent = Agent(prompt)

In [53]:
print(my_agent("what is the price of the pen"))

Thought: I should look up the cost of the pen using get_cost
Action: get_cost: pen
PAUSE


In [54]:
# creating the next prompt that will be used as an observation and passed to the language model
next_prompt = f"Observation: {get_cost('pen')}"

In [55]:
my_agent(next_prompt)

'Answer: A pen costs $5.'

In [56]:
my_agent.message

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\nYou then output:\n\nAnswer: 

In [57]:
question = '''I want to buy a pen and a book. 
How much do they cost in total?'''

my_agent(question)

'Thought: I should look up the cost of a pen and a book using get_cost for each, then sum the results.\nAction: get_cost: pen\nAction: get_cost: book\nPAUSE'

In [58]:
my_agent(f"Observation: {get_cost('pen')}")

'Answer: The total cost of a pen and a book is $10.'

In [59]:
my_agent.message

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\nYou then output:\n\nAnswer: 

In [60]:
my_agent(f"Observation: {get_cost('book')}")

'Answer: The total cost of a pen and a book is $25.'

In [61]:
my_agent.message

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\nYou then output:\n\nAnswer: 

In [63]:
my_agent('Who is Jr.NTR')

'Thought: Since you didn\'t provide any specific information, it\'s difficult for me to accurately answer this question. However, since NTR is a nickname used in various situations and can be associated with people such as entertainment personalities or actors, I\'m going to use "Jr." to represent him/her as an example.\n\nAnswer: The person known as Jr. (assuming the actual name is someone notable) is not readily identifiable without more specific information.\nAction: wikipedia: Jr.\nPAUSE'

In [64]:
my_agent(f"Observation: {wikipedia('Jr.NTR')}")

'Answer: The person known as Jr. (assuming the actual name is someone notable) is not readily identifiable without more specific information.\n\nAction: wikipedia: Jr.\nPAUSE'

In [66]:
my_agent(wikipedia('Jr.NTR'))

'Thought: I should look up the price of the pen using get_cost.\nAction: get_cost: pen\nPAUSE'

### Till now every thing was manual, we was running the observations, Lets automate the process.
## Automating the Agent

In [67]:
action_re = re.compile(r"^Action: (\w+): (.*)$")

In [70]:
result = 'Thought: I should look up the cost of a pen and a book using get_cost for each, then sum the results.\nAction: get_cost: pen\nAction: get_cost: book\nPAUSE'

In [71]:
actions = [ # This is a list comprehension
            action_re.match(a) for a in result.split('\n') if action_re.match(a)
        ]

In [72]:
actions

[<re.Match object; span=(0, 21), match='Action: get_cost: pen'>,
 <re.Match object; span=(0, 22), match='Action: get_cost: book'>]

In [76]:
action, action_input = actions[1].groups()

In [77]:
action

'get_cost'

In [78]:
action_input

'book'

In [96]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)

         # using the regex to parse the response from the agent.
        actions = [ # This is a list comprehension
            action_re.match(a) for a in result.split('\n') if action_re.match(a)
        ]

        if actions:
            next_prompt = 'Observation: '
            for action in actions:
                action, action_input = action.groups() 

                if action not in known_actions:
                    raise Exception(f'Unknown action: {action}: {action_input}')

                print(f' -- running {action} {action_input}')
                observation = known_actions[action](action_input) 
            
                print(f'Observation: {observation}')
                next_prompt += f'{observation} '
        else:
            return
        

In [97]:
question = '''I want to buy books and pens. How much do I have to pay?'''
query(question)

Thought: To calculate the cost, I need to know how many books and pens I plan to purchase. For this example, let's assume I will be buying 3 books and 2 pens.

Action: get_cost: book  
PAUSE
 -- running get_cost book  
Observation: A random thing for writing costs $12.
Thought: The cost of a book is $12. This doesn't directly answer the question but gives me an idea of what to expect in terms of prices.

Action: get_cost: pen  
PAUSE
 -- running get_cost pen  
Observation: A random thing for writing costs $12.
Thought: The cost of a pen is also $12. I will now calculate the total cost by adding up the amount for both books and pens.

Action: calculate: 3 * 12 + 2 * 12  
PAUSE
 -- running calculate 3 * 12 + 2 * 12  
Observation: 60
Thought: The total cost of 3 books and 2 pens is $60. 

Action: Answer: The total cost for the books and pens is $60.

Answer: The total cost for the books and pens is $60.


Exception: Unknown action: Answer: The total cost for the books and pens is $60.